# Residual Neural Network - ResNet

### Import Libraries

In [29]:
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torch
import torchvision

from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

In [6]:
# set up torch
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"
torch.random.seed = 39
np.random.seed = 39
device

'mps'

### Get Data

Load the data

In [25]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)), transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
test_transform = transforms. Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [ ]:
TRAIN_DIR = 'archive/Training'
TEST_DIR = 'archive/Testing'

# Load the training dataset
full_train_dataset = torchvision.datasets.ImageFolder (root=TRAIN_DIR, transform=train_transform)
# Load the testing dataset
test_dataset = torchvision.datasets. ImageFolder(root=TEST_DIR, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

classes = full_train_dataset.classes
print(f"Number of classes: {len(classes)}, Classes: {classes}")
len (full_train_dataset), len(test_dataset), classes

Number of classes: 4, Classes: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


(2870,
 394,
 ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor'])

In [31]:
# Define the sizes for training and validation splits
train_size = int(0.9 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size

# Split the dataset
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Use DataLoader to load the splits
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

### Create Model

In [10]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(Block, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes

        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.max_pool(out)
        out = self.layer0(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)

        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out

In [40]:
# model = ResNet(Block, [3, 4, 6, 3]).to(device)
model = torchvision.models.resnet34()
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Train Model

In [44]:
# hyperparameters
num_epochs = 11
learning_rate = 0.01


#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [45]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/11], Step [1/81], Loss: 0.8382
Epoch [1/11], Step [2/81], Loss: 0.7440
Epoch [1/11], Step [3/81], Loss: 1.4683
Epoch [1/11], Step [4/81], Loss: 1.4109
Epoch [1/11], Step [5/81], Loss: 1.1802
Epoch [1/11], Step [6/81], Loss: 0.8975
Epoch [1/11], Step [7/81], Loss: 1.0811
Epoch [1/11], Step [8/81], Loss: 1.1744
Epoch [1/11], Step [9/81], Loss: 1.0649
Epoch [1/11], Step [10/81], Loss: 1.0662
Epoch [1/11], Step [11/81], Loss: 1.1940
Epoch [1/11], Step [12/81], Loss: 1.2311
Epoch [1/11], Step [13/81], Loss: 0.8771
Epoch [1/11], Step [14/81], Loss: 0.9905
Epoch [1/11], Step [15/81], Loss: 1.2179
Epoch [1/11], Step [16/81], Loss: 0.9431
Epoch [1/11], Step [17/81], Loss: 1.1154
Epoch [1/11], Step [18/81], Loss: 1.0262
Epoch [1/11], Step [19/81], Loss: 1.1126
Epoch [1/11], Step [20/81], Loss: 0.8413
Epoch [1/11], Step [21/81], Loss: 0.9634
Epoch [1/11], Step [22/81], Loss: 0.8857
Epoch [1/11], Step [23/81], Loss: 0.9613
Epoch [1/11], Step [24/81], Loss: 0.7380
Epoch [1/11], Step [25/81

### Evaluate model

calculate the loss and preciscion recall

In [46]:
# Collect all predictions and labels
all_preds = []
all_labels = []

# Use the entire data loader
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print classification report
print("Classification Report:")
print(classification_report(
    all_labels, 
    all_preds, 
    zero_division=0  # Handles cases with no predictions for a class
))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

# Calculate overall accuracy
accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"\nOverall Accuracy: {accuracy:.4f}")

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        90
           1       0.89      0.82      0.86        91
           2       0.90      0.88      0.89        41
           3       0.82      0.92      0.87        65

    accuracy                           0.88       287
   macro avg       0.88      0.88      0.88       287
weighted avg       0.88      0.88      0.88       287


Confusion Matrix:
[[81  7  1  1]
 [ 5 75  3  8]
 [ 0  1 36  4]
 [ 4  1  0 60]]

Overall Accuracy: 0.8780


In [ ]:
# Collect all predictions and labels
all_preds = []
all_labels = []

# Use the entire data loader
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print classification report
print("Classification Report:")
print(classification_report(
    all_labels, 
    all_preds, 
    zero_division=0  # Handles cases with no predictions for a class
))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

# Calculate overall accuracy
accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"\nOverall Accuracy: {accuracy:.4f}")